In [15]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
from pyro import clear_param_store
import pyro.contrib.gp as gp
from pyro.nn import PyroSample
import pyro.distributions as dist
from pyro.infer import MCMC, NUTS, Predictive,HMC
import torch
import kennard_stone as ks
from scipy.optimize import curve_fit
from bokeh.plotting import figure, show
from bokeh.transform import factor_cmap, factor_mark
from bokeh.palettes import Spectral
from bokeh.models import Slope, Div
from bokeh.io import curdoc
from bokeh.layouts import column,gridplot
#from print_versions import print_versions
from sklearn.metrics import r2_score
from seaborn import clustermap
from bokeh.models import Band, ColumnDataSource
from sklearn.neighbors import NearestNeighbors
from sklearn.experimental import enable_iterative_imputer

from sklearn.impute import IterativeImputer

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [16]:
no2=pd.read_excel(r"../data/NO2_VOCs_synthetic.xlsx")


In [17]:
no2=no2.loc[(no2.sample_time<120)&(no2.sample_time>50),:]

In [18]:
data=pd.read_excel(r"../data/pm25_BC_corrected.xlsx")


In [19]:
cpnr=no2.loc[:,["sample_time","corrected_week","gastove","passive","road","sample_time","corrected_week","household"]].groupby("household").mean().reset_index()
X_train, X_test, y_train, y_test = ks.train_test_split(cpnr.loc[:,["gastove","passive","sample_time","corrected_week"]], cpnr.household, test_size = 0.2);
no2=no2.set_index("household")
train=no2.loc[y_train.values,:].reset_index()
test=no2.loc[y_test.values,:].reset_index()


/data/michaelf/miniconda3/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
2025-04-04 12:12:18,132 - kennard_stone.utils._pairwise:109[INFO] - Calculating pairwise distances using scikit-learn.

/data/michaelf/miniconda3/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
2025-04-04 12:12:18,133 - kennard_stone.utils._pairwise:109[INFO] - Calculating pairwise distances using scikit-learn.

/data/michaelf/miniconda3/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [20]:
train_pm,test_pm=data.loc[data.label==0],data.loc[data.label==1]

In [21]:
extra_data_end_pm=train_pm.loc[train_pm.corrected_week<20,:]
extra_data_start_pm=train_pm.loc[train_pm.corrected_week>52.3-20,:]
extra_data_end_pm.corrected_week=extra_data_end_pm.corrected_week+52.3
extra_data_start_pm.corrected_week=-(52.3-extra_data_start_pm.corrected_week)
extra_data_start_pm=extra_data_start_pm.set_index(np.zeros(extra_data_start_pm.shape[0]))
extra_data_end_pm=extra_data_end_pm.set_index(np.zeros(extra_data_end_pm.shape[0]))

/tmp/ipykernel_1206360/1369030321.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extra_data_end_pm.corrected_week=extra_data_end_pm.corrected_week+52.3
/tmp/ipykernel_1206360/1369030321.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extra_data_start_pm.corrected_week=-(52.3-extra_data_start_pm.corrected_week)


In [22]:
extra_data_end=train.loc[train.corrected_week<20,:]
extra_data_start=train.loc[train.corrected_week>52.3-20,:]
extra_data_end.corrected_week=extra_data_end.corrected_week+52.3
extra_data_start.corrected_week=-(52.3-extra_data_start.corrected_week)
extra_data_start=extra_data_start.set_index(np.zeros(extra_data_start.shape[0]))
extra_data_end=extra_data_end.set_index(np.zeros(extra_data_end.shape[0]))

/tmp/ipykernel_1206360/2223900095.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extra_data_end.corrected_week=extra_data_end.corrected_week+52.3
/tmp/ipykernel_1206360/2223900095.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extra_data_start.corrected_week=-(52.3-extra_data_start.corrected_week)


In [23]:
train=pd.concat([train,extra_data_end,extra_data_start])
train_pm=pd.concat([train_pm,extra_data_end_pm,extra_data_start_pm])

In [24]:
train_pm.corrected_week.min()

-19.92051075784304

In [25]:
train.to_excel("../data/chem_train.xlsx")
test.to_excel("../data/chem_test.xlsx")

## PM2.5

In [26]:
train_pm.to_excel("../data/chem_train_pm.xlsx")
test_pm.to_excel("../data/chem_test_pm.xlsx")

In [27]:
train.corr()

,household,Unnamed: 0,maalnr,constrution_year,Area,floor_level,powerplant,road,motorway,forest,...,gastove,fireplace,visit,no2,nox,formaldehyd,acetald,acetone,sample_time,corrected_week
household,1.000000,-0.051479,0.138340,-0.133218,-0.008659,0.121281,0.152614,0.233564,0.246635,-0.079338,...,0.123549,-0.100040,0.257839,0.236228,-0.097522,-0.039534,-0.523315,-0.508825,-0.146555,-0.087792
Unnamed: 0,-0.051479,1.000000,0.047874,0.011692,0.041132,0.010774,0.001888,-0.020483,-0.045610,-0.011555,...,-0.002645,0.018019,0.046247,-0.061276,-0.073890,0.026252,0.012910,-0.012088,0.034161,0.004393
maalnr,0.138340,0.047874,1.000000,0.160607,-0.063695,-0.083564,-0.143134,0.009109,0.093673,0.010039,...,-0.082294,0.090711,0.757786,-0.069286,-0.182583,0.078862,-0.413168,-0.394590,0.097332,-0.105081
constrution_year,-0.133218,0.011692,0.160607,1.000000,-0.145866,-0.145022,-0.368783,-0.348940,-0.003590,0.362516,...,-0.335902,-0.077651,0.117580,-0.354364,-0.090113,0.331585,-0.007485,-0.003723,-0.022061,-0.025649
Area,-0.008659,0.041132,-0.063695,-0.145866,1.000000,-0.261378,-0.063741,-0.229188,-0.086343,-0.012716,...,-0.030958,0.390797,-0.020649,-0.158968,0.040857,-0.033978,-0.020243,0.016896,0.067562,0.032988
floor_level,0.121281,0.010774,-0.083564,-0.145022,-0.261378,1.000000,0.313868,0.281831,0.157456,-0.096068,...,0.261453,-0.088816,-0.101376,0.312780,0.028505,-0.140740,0.036030,-0.079059,-0.059330,-0.017820
powerplant,0.152614,0.001888,-0.143134,-0.368783,-0.063741,0.313868,1.000000,0.269527,0.050661,-0.214460,...,0.547832,-0.058219,-0.099463,0.375637,0.093084,-0.222632,0.049155,0.043622,-0.049873,-0.007572
road,0.233564,-0.020483,0.009109,-0.348940,-0.229188,0.281831,0.269527,1.000000,0.244186,-0.316669,...,0.245934,-0.340988,0.022760,0.359755,0.050671,-0.183297,-0.062114,-0.051902,-0.130983,0.007102
motorway,0.246635,-0.045610,0.093673,-0.003590,-0.086343,0.157456,0.050661,0.244186,1.000000,0.063709,...,0.093748,-0.182671,0.126136,0.094343,0.020665,0.050308,-0.152168,-0.119392,-0.129400,-0.005330
forest,-0.079338,-0.011555,0.010039,0.362516,-0.012716,-0.096068,-0.214460,-0.316669,0.063709,1.000000,...,-0.224007,-0.042881,-0.045779,-0.237428,-0.057856,0.127241,-0.036155,0.081315,-0.031048,0.003925
